In [1]:
import requests as req
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.preprocessing as skp
import sklearn.decomposition as skd
import sklearn.cluster as skc
import gensim

In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True)  

In [106]:
def averageVector(sentence, factor=1):
    _vector = np.zeros(shape=(model.vector_size))
    _c = 0
    for _word in sentence.split(" "):
        if(_word in model.vocab):
            _vector += model.get_vector(_word)
        _c += 1
    return (_vector/_c)/factor

In [185]:
ENTRY = "6qFt3TjvxMt77YGsktWG8Z"
CLIENT = "81e657278a2742739d2020bc48963bf5"
SECRET = "713df9a23eef4999aa75c2515b937b5f"

In [186]:
data = {
    'grant_type': 'client_credentials',
    'client_id': "81e657278a2742739d2020bc48963bf5",
    'client_secret': "713df9a23eef4999aa75c2515b937b5f"
}
res = req.post("https://accounts.spotify.com/api/token", data=data)
token = json.loads(res.text)["access_token"]

In [187]:
songs = {}

In [188]:
headers = {'Authorization': 'Bearer '+token}
nexturl = "https://api.spotify.com/v1/playlists/{}/tracks".format("4k3rMiIW0mgiJI1NBRr4xr")
while nexturl is not None:
    ids = []
    res = json.loads(req.get(nexturl, headers=headers).text)
    nexturl = res["next"]
    for item in res["items"]:
        curr_id = item["track"]["id"]
        ids.append(curr_id)
        songs[curr_id] = {
            "duration": item["track"]["duration_ms"]/60000,
            "explicit": item["track"]["explicit"],
            "name": "{} - {} ({})".format(item["track"]["name"], ",".join([a["name"] for a in item["track"]["artists"]]), item["track"]["album"]["name"]),
            "popularity": item["track"]["popularity"],
            "release_date": int(item["track"]["album"]["release_date"][0:4]),
            "artists_id": ",".join([a["id"] for a in item["track"]["artists"]])
        }
    data = {"ids":",".join(ids)}
    features = json.loads(req.get("https://api.spotify.com/v1/audio-features?ids="+",".join(ids), headers=headers).text)
    for feature in features["audio_features"]:
        songs[feature["id"]].update(feature)
df = pd.DataFrame(songs).T
genres = {}
for artists in np.array_split(list(set(np.hstack(np.array([i.split(",") for i in df["artists_id"].to_list()])))), 50):
    for artist in json.loads(req.get("https://api.spotify.com/v1/artists?ids={}".format(",".join(artists)), headers=headers).text)["artists"]:
        genres[artist["id"]] = " ".join([i for i in artist["genres"]])
artists_id = df["artists_id"].unique()
_g = []
for a in artists_id:
    _g.append({"genres": " ".join([genres[a] for a in a.split(",")]), "artists_id":a})
df = df.merge(pd.DataFrame(_g), left_on='artists_id', right_on='artists_id').drop(["analysis_url", "id", "track_href", "type", "uri", "duration_ms", "artists_id"], axis=1).set_index("name")

embed = {}
for k,v in df["genres"].apply(lambda x : averageVector(x)).iteritems():
    _e =  {}
    for i, vi in enumerate(v):
        _e["e_{}".format(i)] = vi
    embed[k] = _e
    
df = df.drop(["genres"], axis=1)

In [189]:
df.head(130)

,acousticness,danceability,duration,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,release_date,speechiness,tempo,time_signature,valence
name,,,,,,,,,,,,,,,,
N. J Respect R - Damso (Ipséité),0.426,0.747,3.622,0.434,True,0.000405,6,0.108,-8.3,0,2,2017,0.0828,127.052,4,0.311
Θ. Macarena - Damso (Ipséité),0.213,0.777,3.44733,0.549,True,1.73e-06,2,0.0494,-8.202,1,5,2017,0.0614,129.908,4,0.245
Ζ. Kietu - Damso (Ipséité),0.142,0.765,3.87,0.575,True,0,9,0.105,-7.558,0,1,2017,0.105,117.991,4,0.428
Δ. Dieu ne ment jamais - Damso (Ipséité),0.0564,0.767,3.32243,0.734,True,3.35e-06,6,0.0369,-7.787,1,1,2017,0.0467,115.991,4,0.602
Γ. Mosaïque solitaire - Damso (Ipséité),0.37,0.562,5.09177,0.595,True,6.96e-06,5,0.13,-9.896,1,2,2017,0.334,125.172,4,0.426
Α. Nwaar Is The New Black - Damso (Ipséité),0.752,0.744,2.0591,0.604,True,0,2,0.0724,-5.634,0,1,2017,0.15,130.18,4,0.563
Tueurs - Damso (Tueurs),0.666,0.67,3.98692,0.659,True,2.48e-06,2,0.0999,-6.132,0,1,2017,0.114,172.158,4,0.213
Amnésie - Damso (Amnésie),0.75,0.76,3.56048,0.535,True,0,2,0.0922,-9.875,0,0,2017,0.367,127.068,4,0.641
Ε. Signaler - Damso (Ipséité),0.511,0.647,3.36488,0.7,True,3.96e-05,7,0.088,-9.007,0,3,2017,0.0562,176.122,4,0.949


In [192]:
df.columns

Index(['acousticness', 'danceability', 'duration', 'energy', 'explicit',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'popularity',
       'release_date', 'speechiness', 'tempo', 'time_signature', 'valence'],
      dtype='object')

In [193]:
df.popularity.plot.hist(bins=20)

In [194]:
df.sort_values(by="popularity", ascending=False)

,acousticness,danceability,duration,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,release_date,speechiness,tempo,time_signature,valence
name,,,,,,,,,,,,,,,,
"I Love It (& Lil Pump) - Kanye West,Lil Pump (I Love It)",0.0114,0.901,2.13243,0.522,True,0,2,0.259,-8.304,1,78,2018,0.33,104.053,4,0.329
"Médicament - Niska,Booba (Médicament)",0.693,0.665,3.31428,0.653,True,0,6,0.0769,-6.686,0,77,2019,0.0893,157.38,3,0.381
"Maman ne le sait pas (feat. Niska) - Ninho,Niska (Destin)",0.399,0.755,3.38835,0.875,True,0,7,0.115,-5.944,0,75,2019,0.193,145.008,4,0.393
"Tricheur - Nekfeu,Damso (Les étoiles vagabondes : expansion)",0.407,0.758,4.1411,0.638,True,0,1,0.178,-7.984,0,74,2019,0.253,137.981,4,0.188
"Tel Me - Jul,Ninho (Rien 100 Rien)",0.366,0.841,3.32843,0.668,True,1.28e-05,11,0.103,-6.802,0,72,2019,0.16,73.991,4,0.736
Au DD - PNL (Au DD),0.293,0.777,4.11935,0.698,True,2.32e-05,6,0.0908,-7.706,0,71,2019,0.0704,119.963,4,0.119
"Khapta - Heuss L'enfoiré,Sofiane (En esprit)",0.00937,0.811,2.7151,0.692,True,0,4,0.0598,-2.836,0,70,2019,0.105,125.009,4,0.817
"C’est Elle - Alonzo,Ninho (Stone)",0.43,0.812,3.35555,0.607,True,0,7,0.144,-7.919,1,68,2019,0.308,126.058,4,0.255
Est-ce que tu m'aimes ? - Pilule Bleue - Maître Gims (Mon coeur avait raison),0.235,0.779,3.95155,0.852,False,0,2,0.212,-3.353,1,66,2015,0.0651,119.892,4,0.601


In [195]:
df.duration.apply(lambda x: x/60000).plot.hist(bins=20)

In [196]:
df.release_date.plot.hist(bins=50)

In [197]:
df = df.join(pd.DataFrame(embed).T)
scaler = skp.StandardScaler()
scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns, index=df.index)
scaled.head(2)

/home/swnw/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/swnw/.local/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype float64, object were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


,acousticness,danceability,duration,energy,explicit,instrumentalness,key,liveness,loudness,mode,...,e_90,e_91,e_92,e_93,e_94,e_95,e_96,e_97,e_98,e_99
name,,,,,,,,,,,,,,,,,,,,,
N. J Respect R - Damso (Ipséité),0.461710,0.086318,0.100448,-1.820436,0.467426,-0.143910,0.223759,-0.366056,-0.807723,-0.804815,...,0.303421,-0.22199,0.701562,0.201635,-0.114925,0.619017,-0.711159,0.624712,-0.486585,-0.330546
Θ. Macarena - Damso (Ipséité),-0.535484,0.368003,-0.158867,-0.824152,0.467426,-0.202298,-0.983498,-1.035198,-0.749372,1.242521,...,0.303421,-0.22199,0.701562,0.201635,-0.114925,0.619017,-0.711159,0.624712,-0.486585,-0.330546


In [198]:
clusterized = df.copy()
kmeans = skc.KMeans(n_clusters=5, random_state=0).fit(scaled)
clusterized["cluster"] = kmeans.labels_
playlists = clusterized.groupby("cluster")

In [199]:
for i, p in playlists:
    print("Playlist {}".format(i))
    for song in p.T:
        print("  "+song[:80])

Playlist 0
  Afro Trap Pt. 7 (La puissance) - MHD (Afro Trap Part. 7 (La puissance))
  Afro Trap Pt. 8 (Never) - MHD (Afro Trap Part. 8 (Never))
  Encore - MHD (19)
  Rouler - MHD (19)
  Bravo - MHD (19)
  Fuego - MHD (19)
  XIX - MHD (19)
  Réseaux - Niska (Commando)
  Salé - Niska (Commando)
  Medellín - Niska (Commando)
  Okkk - Sneazzy,S.Pri Noir (Dieu bénisse Supersound)
  Évite la - Sneazzy,S.Pri Noir (Dieu bénisse Supersound, saison 2)
  MAC 11 - SCH (Deo Favente)
  Fusil - SCH (A7)
  Comme si - SCH (Deo Favente)
  6.45i - SCH (Deo Favente)
  La nuit - SCH (Deo Favente)
  Anarchie - SCH (Anarchie)
  A7 - SCH (A7)
  Poupée russe - SCH (Deo Favente)
  Booska Favente - SCH (Booska Pefra, Vol. 4)
  Mort de rire - SCH (Mort de rire)
  Otto - SCH (Otto)
  Ivresse & Hennessy - SCH (JVLIVS)
  Pharmacie - SCH (JVLIVS)
  Tokarev - SCH (JVLIVS)
  Ciel rouge - SCH (JVLIVS)
  Le code - SCH (JVLIVS)
  Haut standing - SCH (Haut standing)
  L’art et la manière - S.Crew,Doums (Destins liés)
  Oi

In [179]:
pca = skd.PCA(n_components=3)
pca.fit(scaled)
transformed = pd.DataFrame(pca.transform(scaled))
transformed.head(4)

,0,1,2
0,17.569973,-15.240387,2.744118
1,17.665116,-15.161805,2.700179
2,17.594822,-15.254420,2.831480
3,17.715629,-15.091411,2.696202
